In [24]:
from utils.LSTM import Long_Short_Term_Memory, company_data
from utils.Monte_Carlo import MonteCarlo, get_company_data
from datetime import datetime as dt
import numpy as np
import pandas as pd

#### Monte Carlo to determine portfolio

In [ ]:
start_fund = 100
# obtain portfolio
end_time = dt(2019, 7, 1)
start_time = dt(2004, 1, 1)
company_list = ['A', 'AA', 'ABC', 'ABCB', 'ACLS','ACNB', 'ADBE', 'ADP', 'AEG', 'AIR']
stock_price, start_price = [], []
for company_name in company_list:
    company_data = get_company_data(company_name, start_time, end_time)
    start_price.append(company_data.tail(1)['Close'].values)
    # compute fractional change of current and previous for measuring changes
    # which is given as (next - previous) / previous
    stock_data = np.array(company_data['Close'].pct_change().dropna())
    stock_price.append(stock_data)
price_array = np.array(stock_price)
price_series = pd.DataFrame(price_array.T, columns=company_list)
start_price = np.array(start_price).flatten()
# monte-carlo simulation
model = MonteCarlo(price_series)
start_fund = 100
portfolio = model.build_portfolio()
company_dict = {}
for company in company_list:
    company_dict[company] = portfolio[company_list.index(company)] * start_fund
company_dict

#### LSTM to predict prices

In [25]:
data = pd.read_csv('..\\data\\2005_2019_10_close.csv', parse_dates=['Date'])
# convert date to time datetime
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
data = pd.read_csv('..\\data\\2005_2019_10_close.csv', parse_dates=['Date'])
# convert date to time datetime
data['Date'] = pd.to_datetime(data['Date'])
company = 'AA'
start_time = dt(2014,1,1)
end_time = dt(2019,1,1)
company_stock_price = company_data(data, company, start_time, end_time)
price = np.array(company_stock_price[company])
model = Long_Short_Term_Memory(price, test_percent=0.2, seq_length=60)
model.fit(restore_weights=True, restore_path='..\\weights\\LSTM_AA_weights.h5')
test_MAE = model.evaluate()
predicted = model.predict()
print(f"The test error: {test_MAE} \n \
    The predicted value: {predicted}")

In [ ]:
start_time = dt(2019, 3, 30)
end_time = dt(2019, 9, 1)
time_data = company_data(data, company, start_time, end_time)
AA_data = np.array(time_data[company])
input_data = np.array([[each] for each in AA_data])
input_data = input_data.reshape(input_data.shape[0], input_data.shape[1], 1)
y_predict = model.predict(input_data)
y_predict

#### Load y predict and y true from files

In [58]:
predict_true = pd.read_csv('..\\data\\AA_predictions.csv')
y_predict = np.array(predict_true['Predictions'])
true_price = np.array(predict_true['Actual'])

In [ ]:
totalFund = 50
extraFund = 50 # split original fund for dynamic processing
price_at_window_start, price_at_window_end = totalFund, 0
stockNum = 0 # current number of stock at hand
dayNum = len(y_predict)
has_stock = False # has stock in hand currently
for today in range(0, dayNum - 1): # take one day in advance
    if y_predict[today + 1] > true_price[today] and not has_stock:
        # stock predicted to be rising and dont have stock yet
        price_at_window_start = totalFund
        stockNum = totalFund / true_price[today]
        totalFund = 0
        has_stock = True
    elif y_predict[today + 1] < true_price[today] and has_stock:
        # price predicted to be falling, sell the stock
        totalFund = stockNum * true_price[today]
        has_stock = False
        # examine the stock
        price_at_window_end = totalFund
        ratio = price_at_window_start / price_at_window_end
        if ratio < 1:
            # promising stock, add more fund
            increment = totalFund * (1 - ratio)
            if extraFund < increment:
                extraFund = 0
                totalFund += extraFund
            else:
                extraFund -= increment
                totalFund += increment
        else:
            # reduce the fund in this stock
            decrement = totalFund * (ratio - 1)
            totalFund -= decrement
            extraFund += decrement
        print(f"Sold stock at day{today}, \n \
        totalFund: {totalFund}, extraFund: {extraFund}")